### dim_TipoPagamento

In [0]:
%sql
USE SCHEMA olist;

CREATE TABLE IF NOT EXISTS dim_TipoPagamento (
  id BIGINT GENERATED ALWAYS AS IDENTITY,
  nm_tipo_pagamento STRING
)
USING DELTA;


In [0]:

from delta.tables import DeltaTable

df_tipo_pagamento = spark.sql("""
    SELECT DISTINCT tipo_pagamento_formatado
    FROM delta.`dbfs:/mnt/olist/Silver/olist_order_payments_dataset`
""")

df_dim_pagamento = DeltaTable.forName(spark, "olist.dim_TipoPagamento")

df_dim_pagamento.alias("gold").merge(df_tipo_pagamento.alias("silver"),"gold.nm_tipo_pagamento = silver.tipo_pagamento_formatado"
        ).whenNotMatchedInsert(
    values={"nm_tipo_pagamento": "silver.tipo_pagamento_formatado"}
).execute()

#display(display(spark.read.table("olist.dim_TipoPagamento")))



### dim_Produto

In [0]:
%sql
USE SCHEMA olist;

CREATE TABLE IF NOT EXISTS dim_CategoriaProduto (
  id BIGINT GENERATED ALWAYS AS IDENTITY,
  nm_categoria_produto STRING
)
USING DELTA;

In [0]:
from delta.tables import DeltaTable

df_categoria_produto = spark.sql('''
    select distinct 
    nome_categoria_produto_formatado
FROM delta.`dbfs:/mnt/olist/Silver/olist_products_dataset`
    where nome_categoria_produto_formatado is not null
''')

df_dim_categoria_produto = DeltaTable.forName(spark, "olist.dim_CategoriaProduto")

df_dim_categoria_produto.alias("gold").merge(df_categoria_produto.alias("silver"),"gold.nm_categoria_produto = silver.nome_categoria_produto_formatado"
        ).whenNotMatchedInsert(
    values={"nm_categoria_produto": "silver.nome_categoria_produto_formatado"}
).execute()

#display(display(spark.read.table("olist.dim_CategoriaProduto")))



### dim_StatusPedido

In [0]:
%sql
USE SCHEMA olist;

CREATE TABLE IF NOT EXISTS dim_StatusPedido (
  id BIGINT GENERATED ALWAYS AS IDENTITY,
  nm_status_pedido STRING
)
USING DELTA;

In [0]:
from delta.tables import DeltaTable

df_status_pedido = spark.sql('''
select distinct
  status_pedido_formatado

from delta.`dbfs:/mnt/olist/Silver/olist_orders_dataset`
''')

df_dim_status_pedido = DeltaTable.forName(spark, "olist.dim_StatusPedido")

df_dim_status_pedido.alias("gold").merge(df_status_pedido.alias("silver"),"gold.nm_status_pedido = silver.status_pedido_formatado"
        ).whenNotMatchedInsert(
    values={"nm_status_pedido": "silver.status_pedido_formatado"}
).execute()

#display(display(spark.read.table("olist.dim_StatusPedido")))


### dim_vendendor

In [0]:
%sql
USE SCHEMA olist;


CREATE TABLE IF NOT EXISTS dim_vendedor (
  id BIGINT GENERATED ALWAYS AS IDENTITY
  ,codigo STRING
  ,matricula STRING
  ,estado STRING
)
USING DELTA;

In [0]:
from delta.tables import DeltaTable

df_vendedor = spark.sql('''
select distinct
   vendedor_id
  ,concat(left(vendedor_id,2), lpad(cep_prefixo_vendedor,5,0)) matricula
  ,estado_vendedor

from delta.`dbfs:/mnt/olist/Silver/olist_sellers_dataset`
''')

df_dim_vendedor = DeltaTable.forName(spark, "olist.dim_vendedor")

df_dim_vendedor.alias("gold").merge(df_vendedor.alias("silver"),"gold.codigo = silver.vendedor_id"
        ).whenNotMatchedInsert(
    values={"matricula": "silver.matricula"
            ,"codigo": "silver.vendedor_id"
            ,"estado": "silver.estado_vendedor"}
).execute()

#display(display(spark.read.table("olist.dim_vendedor")))



### fato_vendas

In [0]:
%sql
USE SCHEMA olist;


CREATE TABLE IF NOT EXISTS fato_vendas (
   sk_tempo INTEGER
  ,sk_TipoPagamento INTEGER
  ,sk_TipoCategoriaProduto INTEGER
  ,Estado STRING
  ,sk_StatusPedido INTEGER
  ,sk_vendedor INTEGER
  ,qtd_pedidos INTEGER
  ,qtd_itens INTEGER
  ,vl_faturamento decimal(15,2)
  ,qtd_clientes INTEGER
  ,total_avaliacao_score INTEGER
)
USING DELTA;

In [0]:
from delta.tables import DeltaTable

df_vendas = spark.sql('''
SELECT 
  concat(year(data_compra_pedido),lpad(month(data_compra_pedido),2,'0'),'01') sk_tempo   
  ,tp.id sk_TipoPagamento
  ,coalesce(cp.id,77) sk_TipoCategoriaProduto
  ,c.Estado
  ,sp.id sk_StatusPedido
  ,v.id sk_vendedor
  ,count(o.pedido_id) qtd_pedidos
  ,sum(i.pedido_item_id) qtd_itens
  ,sum(i.preco * i.pedido_item_id) vl_faturamento
  ,count(distinct o.cliente_id) qtd_clientes
  ,sum(r.avaliacao_score) total_avaliacao_score
 
FROM delta.`dbfs:/mnt/olist/Silver/olist_orders_dataset` as o
  join delta.`dbfs:/mnt/olist/Silver/olist_order_items_dataset` as i on o.pedido_id = i.pedido_id
  join delta.`dbfs:/mnt/olist/Silver/olist_order_reviews_dataset` as r on o.pedido_id = r.pedido_id
  join delta.`dbfs:/mnt/olist/Silver/olist_order_payments_dataset` as p on o.pedido_id = p.pedido_id
  join delta.`dbfs:/mnt/olist/Silver/olist_products_dataset` as pr on i.produto_id = pr.produto_id
  left join olist.dim_CategoriaProduto as cp on cp.nm_categoria_produto = pr.nome_categoria_produto_formatado
  join olist.dim_TipoPagamento as tp on tp.nm_tipo_pagamento = p.tipo_pagamento_formatado
  join delta.`dbfs:/mnt/olist/Silver/olist_customers_dataset` as c on c.cliente_id = o.cliente_id
  Join olist.dim_StatusPedido as sp on sp.nm_status_pedido = o.status_pedido_formatado
  join olist.dim_vendedor as v on v.codigo = i.vendedor_id
group by 
  concat(year(data_compra_pedido),lpad(month(data_compra_pedido),2,'0'),'01')
  ,tp.id
  ,coalesce(cp.id,77)
  ,c.Estado
  ,sp.id 
  ,v.id 
''')

df_fato_vendas = DeltaTable.forName(spark, "olist.fato_vendas")

df_fato_vendas.alias("gold").merge(df_vendas.alias("silver"),"gold.sk_tempo = silver.sk_tempo AND gold.sk_TipoPagamento = silver.sk_TipoPagamento AND gold.sk_TipoCategoriaProduto = silver.sk_TipoCategoriaProduto AND gold.Estado = silver.Estado AND gold.sk_StatusPedido = silver.sk_StatusPedido AND gold.sk_vendedor = silver.sk_vendedor"
        ).whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()

display(display(spark.read.table("olist.fato_vendas")))





In [0]:
%sql
USE CATALOG hive_metastore;
USE SCHEMA olist;
select max(sk_tempo), min(sk_tempo) from olist.fato_vendas